Scrapping data from the minfin.gov website

In [1]:
# .\env\Scripts\activate
#!pip install altair
import os
import requests
import pandas as pd
from urllib.request import Request, urlopen
import warnings
from pathlib import Path
import altair as alt
warnings.filterwarnings('ignore')

In [2]:
req = Request(url='https://minfin.gov.ru/ru/perfomance/nationalwealthfund/statistics?id_57=27068-obem_fonda_natsionalnogo_blagosostoyaniya', headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
df_list = pd.read_html(webpage)
df = pd.DataFrame(df_list[0])
df = df.rename(columns={df.columns[0]: "Date", df.columns[1]: "amount_blnUSD",df.columns[2]: 'amount_blnRUB'})
df = df.drop(columns=df.columns[3])
df['amount_blnRUB'] = df['amount_blnRUB'].str.replace(r'\D+', '')
df['amount_blnRUB'] = df['amount_blnRUB'].astype(int) / 100
df['amount_blnUSD'] = df['amount_blnUSD'].astype(int) / 100
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')
df['Date(m,Y)'] = df['Date'].dt.strftime('%B, %Y')
print('Data downloaded')

Data downloaded


Saving data into CSV file

In [3]:
filepath = Path('C:/Users/lenovo/python/FNB/FNB/fnb.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)
print('CSV file saved')

CSV file saved


Plotting the data

In [7]:
datetime_invasion = pd.to_datetime('24.02.2022', format='%d.%m.%Y')
line = alt.Chart(df).mark_line(point=True).encode(
    x=alt.X("Date(m,Y):T", axis=alt.Axis(title=None, format=("%b %Y"), labelAngle=90)), 
    y=alt.Y('amount_blnUSD:Q'), 
    tooltip = ["Date(m,Y):T", 'amount_blnUSD:Q'])
rule = alt.Chart(pd.DataFrame({'datetime_invasion': [datetime_invasion]})).mark_rule(color='red').encode(x='datetime_invasion:T')
(line + rule).properties(title='Fund amount in USD', width=1300)

alt.LayerChart(...)

In [6]:
line = alt.Chart(df).mark_line(point=True).encode(
    x=alt.X("Date(m,Y):T", axis=alt.Axis(title=None, format=("%b %Y"), labelAngle=90)), 
    y=alt.Y('amount_blnRUB:Q'),
    tooltip = ["Date(m,Y):T", 'amount_blnRUB:Q'])
rule = alt.Chart(pd.DataFrame({'datetime_invasion': [datetime_invasion]})).mark_rule(color='red').encode(x='datetime_invasion:T')
(line + rule).properties(title='Fund amount in RUB', width=1300)

alt.LayerChart(...)

Downloadind monthly info messages from Minfin:

In [ ]:
def download_file(url, save_path):
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)

base_url = 'https://minfin.gov.ru/common/upload/library'
output_folder = 'data'

for year in range(2008, 2023):  # 2023 not included in the range
    for month in range(1, 13):  
        date_str = f'01.{month:02d}.{year}'
        url = f'{base_url}/{year}/{month:02d}/main/{date_str}.docx'
        save_path = os.path.join(output_folder, f'{year}_{month:02d}.docx')
        download_file(url, save_path)
        print(f'Downloaded: {url}')

The result is 15 donloaded .docx files from the February 2019 until June 2023. But in fact messages regarding FNB fund are only the beginning of 2023 year.

Different file names for different periods:
* https://minfin.gov.ru/common/upload/library/2021/11/main/01.11.2021.docx
* https://minfin.gov.ru/common/upload/library/2019/09/main/fnb_01.09.2019.docx
* https://minfin.gov.ru/common/upload/library/2018/05/main/FNB_-_na_01.05.2018.docx
* https://minfin.gov.ru/common/upload/library/2017/06/main/Informatsionnoe_soobshchenie_o_rezultatakh_razmeshcheniya_sredstv_RF_i_FNB_-_na_01.06.2017.docx
* https://minfin.gov.ru/common/upload/library/2016/07/main/Informatsionnoe_soobshchenie_o_rezultatakh_razmeshcheniya_sredstv_Rezervnogo_fonda_i_FNB_01_07_2016.docx
* https://minfin.gov.ru/common/upload/library/2015/11/main/Informatsionnoe_soobshchenie_02_11_2015.doc

In [114]:
liquid = pd.read_excel('FNB_structure.xlsx', sheet_name='Liquid', header=None, index_col=0)
liquid = liquid.T
liquid

,Dates,"Funds_on_the_Bank_of_Russia_accounts,mln_RUB","Funds_on_the_Bank_of_Russia_accounts,mln_USD",Share_of_liquid_funds_in_the_total_Fund_in_US_dollar_equivalent
1,2022-10-01,7524361.3,131056.8,0.697205
2,2022-11-01,7872548.3,127937.6,0.692148
3,2022-12-01,7601763,124467.7,0.667436
4,2023-01-01,6132655.2,87189,0.587724
5,2023-02-01,6334590.6,91023.8,0.586124
6,2023-03-01,6446231,85457.2,0.580407
7,2023-04-01,6712063.4,87072.1,0.563752
8,2023-05-01,6828671.4,84818.4,0.547363
9,2023-06-01,6643004.4,82330.3,0.537758
10,2023-07-01,6811125,78258.1,0.537567


In [118]:
line = alt.Chart(liquid).mark_line(point=True).encode(
    x=alt.X("Dates:T"),# axis=alt.Axis(title=None, format=("%b %Y"), labelAngle=90)), 
    y=alt.Y('Funds_on_the_Bank_of_Russia_accounts,mln_RUB:Q'),
    tooltip = ["Dates:T", 'Funds_on_the_Bank_of_Russia_accounts,mln_RUB:Q']).properties(title='Liquid Fund amount in RUB', width=1000)
line

alt.Chart(...)

In [119]:
line = alt.Chart(liquid).mark_line(point=True).encode(
    x=alt.X("Dates:T"),# axis=alt.Axis(title=None, format=("%b %Y"), labelAngle=90)), 
    y=alt.Y('Funds_on_the_Bank_of_Russia_accounts,mln_USD:Q'),
    tooltip = ["Dates:T", 'Funds_on_the_Bank_of_Russia_accounts,mln_USD:Q']).properties(title='Liquid Fund amount in USD', width=1000)
line

alt.Chart(...)